In [1]:
#Library imports
import re
import os
import sys
import shutil
import time
import math
import fnmatch
import random
import warnings
import numpy as np
import pandas as pd
import scipy.stats as scs
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as scs

import fiona
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from pyproj import Proj
import geoplot as gplt
import geoplot.crs as gcrs

# Allows access to scripts and modules relative to the parent directory.
parent = os.getcwd()
sys.path.append(os.path.join(parent, "functions"))

# Project specific user driven functions
from cleaning_functions import *

# My open source reusable user driven function repository.
from random_lumberjacks.src.random_lumberjacks.cleaning.cleaning_functions import *
from random_lumberjacks.src.random_lumberjacks.model.model_classes import *
from random_lumberjacks.src.random_lumberjacks.visualization.visualization_functions import *

#Notebook arguments
%matplotlib inline

In [ ]:
water_final = pd.read_csv("data/Water_FINAL.csv", low_memory = False)
water_dict = pd.read_csv("data/CBP_water_dictionary_final.csv")

In [3]:
water_final.head()

,Unnamed: 0,Unnamed: 0.1,Agency,BiasPC,CloudCover,Comments,Cruise,Database,Date,FieldActivityEventType,...,Weather Conditions Yesterday,WindDirection,WindSpeed,state,Point,HUC12_,HUCNAME_,FIPS_,COUNTY_,STATE_
0,0,2143,NaN,NaN,NaN,NaN,NaN,CMC,04/28/2018,NaN,...,NaN,W,1-10 knots,MD,POINT (-75.93002800000002 39.480194),20600020206,Bohemia River,24015,Cecil County,MD
1,1,2145,NaN,NaN,NaN,NaN,NaN,CMC,04/28/2018,NaN,...,NaN,W,1-10 knots,MD,POINT (-75.902528 39.480444),20600020206,Bohemia River,24015,Cecil County,MD
2,2,2147,NaN,NaN,NaN,NaN,NaN,CMC,04/28/2018,NaN,...,NaN,S,1-10 knots,MD,POINT (-75.873639 39.466889),20600020206,Bohemia River,24015,Cecil County,MD
3,3,2151,NaN,NaN,NaN,NaN,NaN,CMC,04/28/2018,NaN,...,NaN,SW,1-10 knots,MD,POINT (-75.854444 39.454),20600020206,Bohemia River,24015,Cecil County,MD
4,4,2152,NaN,NaN,NaN,NaN,NaN,CMC,04/28/2018,NaN,...,NaN,SW,1-10 knots,MD,POINT (-75.84352800000001 39.463667),20600020206,Bohemia River,24015,Cecil County,MD


In [5]:
df = water_final.copy()

In [6]:
def preliminary_cleaning(df):
    
    # Combining the Date and Time parameters into a single Datetime Object
    df["Date"] = df["Date"]+"-"+df["Time"]
    df["Date"] = pd.to_datetime(df["Date"])
    
    # The Station identifiers are unique between the two datasets so they can be merged into a single column.
    df['Station'] = np.where(df['Station'].isnull(), df['StationCode'], df['Station'])
    
    # It will be easier to compare observed tide data by combining them into the same column first.
    df["Tidal Stage"] = df["Tidal Stage"].map(standardize_CMC_tidal_strings)
    df['TideStage'] = np.where(df['TideStage'].isnull(), df['Tidal Stage'], df['TideStage'])
    
    # Regex cleanup of weather conditions field.
    df["Weather Conditions Today"]=df["Weather Conditions Today"].map(standardize_CMC_weather_strings)
    df["Weather Conditions Yesterday"]=df["Weather Conditions Yesterday"].map(standardize_CMC_weather_strings)
    
    # Regex cleanup of wind speed field.
    df["WindSpeed"] = df["WindSpeed"].map(standardize_CMC_wind_strings)
    
    # Regex cleanup of the Water Surfaces field
    df["Water Surfaces"] = df["Water Surfaces"].map(standardize_CMC_water_surf_strings)
    df.drop(columns = ["Unnamed: 0", "Unnamed: 0.1", "FieldActivityRemark", "PrecisionPC", "Comments",
                       "Time", "Other Comments", "Other Conditions", "StationCode", "StationName",
                       "Tidal Stage", "ModifiedDate", "Water Color", "Water Color Description",
                       "Water Odor", "Water Odor Description", "state", 'COUNTY_', 'STATE_'], inplace=True)
preliminary_cleaning(df)

In [13]:
#Calls a user created function to find unique/overlapping naming conventions in the CMC/CBD data to better unify them.
full_value_comparison(df, "ParameterName_CBP", "Parameter")

ACTIVE CHLOROPHYLL-A
There are 4 unique values with 0 nan values making up 0.0%
98506-CHLA --98.68%
1137-CHL.2 --1.14%
137-CHL.3 --0.14%
40-CHL.4 --0.04%

DO SATURATION USING PROBE UNITS IN PERCENT
There are 3 unique values with 0 nan values making up 0.0%
8651-DO.3 --73.42%
3050-DO.6 --25.88%
82-DO.15 --0.70%

Not in CBP Database
There are 21 unique values with 0 nan values making up 0.0%
10779-PH.4 --35.54%
3706-NO3N.1 --12.22%
3454-ECOLI.1 --11.39%
3326-CHL.1 --10.97%
2366-OP.7 --7.80%
1373-ENT.2 --4.53%
1309-OP.1 --4.32%
978-SA.1 --3.22%
792-NO3N.5 --2.61%
642-ECOLI.6 --2.12%
302-SA.2 --1.00%
280-ECOLI.2 --0.92%
269-NO2NO3.4 --0.89%
205-NO3N.4 --0.68%
172-NO2NO3.1 --0.57%
144-TDS.3 --0.47%
93-PH.8 --0.31%
91-ENT.1 --0.30%
29-WC.12 --0.10%
15-OP.4 --0.05%
1-PH.5 --0.00%

SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU)
There are 6 unique values with 0 nan values making up 0.0%
357455-SALINITY --96.24%
8437-SA.3 --2.27%
2925-SA.6 --0.79%
1549-SA.9 --0.42%
834-SA.10 --

In [ ]:
full_value_counts(df, "Parameter")

In [19]:
df[df["ParameterName_CMC"]=="Salinity "]['Parameter'].head()

3979    SA.9
3980    SA.9
3981    SA.9
3982    SA.9
3983    SA.9
Name: Parameter, dtype: object

In [27]:
full_value_counts(df, "ParameterName_CMC")

There are 27 unique values with 2444456 nan values making up 90.2%
39722-Water temperature  --1.47%
38319-Dissolved oxygen  --1.41%
35736-pH  --1.32%
21706-Conductivity  --0.80%
20144-Water Clarity  --0.74%
16599-Air temperature  --0.61%
15252-Salinity  --0.56%
11701-Dissolved oxygen % Sat  --0.43%
11124-Nitrate-nitrogen  --0.41%
10442-Total Dissolved Solids  --0.39%
8834-Orthophosphate  --0.33%
6989-Bacteria [E.Coli]  --0.26%
6418-Alkalinity  --0.24%
5052-Total Depth  --0.19%
4463-Chlorophyll a  --0.16%
3066-Total Phosphorus  --0.11%
2610-Total Nitrogen  --0.10%
2351-Water Temperature  --0.09%
1464-Enterococcus  --0.05%
1044-Total Suspended Solids  --0.04%
441-Nitrite-nitrate  --0.02%
292-Ammonia-nitrogen  --0.01%
177-Chlorophyll a;b;c  --0.01%
125-Air Temperature  --0.00%
108-Total Kjeldahl Nitrogen  --0.00%
82-Dissolved Oxygen % Sat  --0.00%


In [12]:
full_value_counts(df, "ParameterName_CBP")

There are 24 unique values with 0 nan values making up 0.0%
430638-WATER TEMPERATURE DEG --15.90%
423900-DISSOLVED OXYGEN IN MG/L --15.65%
411264-PH CORRECTED FOR TEMPERATURE (25 DEG C) --15.18%
371427-SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU) --13.71%
128083-TOTAL SUSPENDED SOLIDS --4.73%
122577-ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE) --4.53%
121419-NITRATE NITROGEN AS N (FILTERED SAMPLE) --4.48%
114674-TOTAL PHOSPHORUS --4.23%
112110-TOTAL NITROGEN --4.14%
111958-AMMONIUM NITROGEN AS N (FILTERED SAMPLE) --4.13%
99820-ACTIVE CHLOROPHYLL-A --3.69%
56980-SECCHI DEPTH --2.10%
43796-TURBIDITY; NEPHELOMETRIC METHOD --1.62%
30326-Not in CBP Database --1.12%
28749-TOTAL ALKALINITY AS CACO3 --1.06%
21706-SPECIFIC CONDUCTIVITY --0.80%
17935-TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C --0.66%
16724-AIR TEMPERATURE DEG --0.62%
14365-TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE) --0.53%
11783-DO SATURATION USING PROBE UNITS IN PERCENT --0.44%
7620-TURBIDITY TUBE -

In [33]:
df["Database"].value_counts().index

Index(['CBP', 'CMC'], dtype='object')

In [26]:
df.columns

Index(['Agency', 'BiasPC', 'CloudCover', 'Comments', 'Cruise', 'Database',
       'Date', 'FieldActivityEventType', 'FlowStage', 'GaugeHeight',
       'GroupCode', 'HUC12', 'Lab', 'Latitude', 'Layer', 'Longitude',
       'LowerPycnocline', 'MeasureValue', 'Method', 'ModifiedDate',
       'Other Comments', 'Other Conditions', 'Parameter', 'ParameterName_CBP',
       'ParameterName_CMC', 'PrecipType', 'Pressure', 'Problem', 'Program',
       'Project', 'Qualifier', 'Rainfall', 'Rainfall Within 24 Hours',
       'Rainfall Within 48 Hours', 'SampleDepth', 'SampleId',
       'SampleReplicateType', 'SampleType', 'Sea State', 'Source', 'Station',
       'StationCode', 'StationName', 'Stream Flow', 'Tidal Stage', 'TideStage',
       'TierLevel', 'Time', 'TotalDepth', 'Unit', 'UpperPycnocline',
       'Water Color', 'Water Color Description', 'Water Odor',
       'Water Odor Description', 'Water Surfaces', 'WaveHeight',
       'Weather Conditions Day Before Yesterday', 'Weather Conditions Today

In [5]:
ref

,CBP Code,CBP Parameter,CBP Unit,CMC Parameter,CMC Codes,Notes
0,ATEMP,AIR TEMPERATURE DEG,C,Air Temperature,"AT.1, AT.2, AT.3, AT.4, AT.5, AT.6, AT.8, AT.9...",NaN
1,CHLA,ACTIVE CHLOROPHYLL-A,UG/L,Chlorophyll A,"CHL.2, CHL.3, CHL.4",NaN
2,DO,DISSOLVED OXYGEN IN MG/L,MG/L,Dissolved Oxygen,"DO.4, DO.14, DO.1, DO.8, DO.9, DO.5, DO.2, DO.7",NaN
3,DO_SAT_P,DO SATURATION USING PROBE UNITS IN PERCENT,PCT,Dissolved Oxygen % Sat,"DO.15, DO.3, DO.6",NaN
4,NH4F,AMMONIUM NITROGEN AS N (FILTERED SAMPLE),MG/L,Ammonia-nitrogen,NH4N.1,NaN
5,NO3F,NITRATE NITROGEN AS N (FILTERED SAMPLE),MG/L,Nitrate-nitrogen,"NO3N.3, NO3N.6, NO3N.2","NO3N.1, NO3N.5 - not included in this paramete..."
6,PH,PH CORRECTED FOR TEMPERATURE (25 DEG C),SU,pH,"PH.3, PH.10, PH.6, PH.7, PH.2, PH.1, PH.9, PH.11",NaN
7,PO4F,ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE),MG/L,Orthophosphate,"OP.8, OP.6, OP.3, OP.2","OP.1, OP.7 - not included in this parameter an..."
8,SALINITY,SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL S...,PPT,Salinity,"SA.3, SA.8, SA.6, SA.10, SA.9","SA.1, SA.2 - not included in this parameter an..."
9,SECCHI,SECCHI DEPTH,M,Water Clarity,"WC.1, WC.2, WC.7",NaN


In [17]:
print_full(df.iloc[0])

Agency                                                                      NaN
BiasPC                                                                      NaN
CloudCover                                                                  NaN
Cruise                                                                      NaN
Database                                                                    CMC
Date                                                        2018-04-28 12:05:00
FieldActivityEventType                                                      NaN
FlowStage                                                                   NaN
GaugeHeight                                                                 NaN
GroupCode                                                                    FB
HUC12                                                                       NaN
Lab                                                                         NaN
Latitude                                

In [16]:
full_value_counts(df, "Sea State")

There are 8 unique values with 2703287 nan values making up 99.8%
1906-<1 foot --0.07%
1529-Calm --0.06%
1183-<1 Foot --0.04%
469-<2 feet --0.02%
263-calm --0.01%
43-<2 Feet --0.00%
37-<3 feet --0.00%


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708717 entries, 0 to 2708716
Data columns (total 71 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   Agency                                   object        
 1   BiasPC                                   object        
 2   CloudCover                               object        
 3   Comments                                 object        
 4   Cruise                                   object        
 5   Database                                 object        
 6   Date                                     datetime64[ns]
 7   FieldActivityEventType                   object        
 8   FieldActivityRemark                      float64       
 9   FlowStage                                object        
 10  GaugeHeight                              float64       
 11  GroupCode                                object        
 12  HUC12                       

In [53]:
df.columns

Index(['Agency', 'BiasPC', 'CloudCover', 'Comments', 'Cruise', 'Database',
       'Date', 'FieldActivityEventType', 'FlowStage', 'GaugeHeight',
       'GroupCode', 'HUC12', 'Lab', 'Latitude', 'Layer', 'Longitude',
       'LowerPycnocline', 'MeasureValue', 'Method', 'ModifiedDate',
       'Other Comments', 'Other Conditions', 'Parameter', 'ParameterName_CBP',
       'ParameterName_CMC', 'PrecipType', 'Pressure', 'Problem', 'Program',
       'Project', 'Qualifier', 'Rainfall', 'Rainfall Within 24 Hours',
       'Rainfall Within 48 Hours', 'SampleDepth', 'SampleId',
       'SampleReplicateType', 'SampleType', 'Sea State', 'Source', 'Station',
       'StationCode', 'StationName', 'Stream Flow', 'Tidal Stage', 'TideStage',
       'TierLevel', 'Time', 'TotalDepth', 'Unit', 'UpperPycnocline',
       'Water Color', 'Water Color Description', 'Water Odor',
       'Water Odor Description', 'Water Surfaces', 'WaveHeight',
       'Weather Conditions Day Before Yesterday', 'Weather Conditions Today

In [40]:
df.sort_values("Date")

,Agency,BiasPC,CloudCover,Comments,Cruise,Database,Date,FieldActivityEventType,FlowStage,GaugeHeight,...,Weather Conditions Yesterday,WindDirection,WindSpeed,state,Point,HUC12_,HUCNAME_,FIPS_,COUNTY_,STATE_
61127,NaN,NaN,NaN,NaN,NaN,CMC,1992-11-01,NaN,NaN,NaN,...,NaN,NaN,NaN,PA,POINT (-77.18104 40.20792),20503050404,Letort Spring Run,42041,Cumberland County,PA
211099,NaN,NaN,NaN,NaN,NaN,CMC,1992-11-01,NaN,NaN,NaN,...,NaN,NaN,NaN,PA,POINT (-77.18423 40.19528),20503050404,Letort Spring Run,42041,Cumberland County,PA
211100,NaN,NaN,NaN,NaN,NaN,CMC,1992-11-01,NaN,NaN,NaN,...,NaN,NaN,NaN,PA,POINT (-77.18104 40.20792),20503050404,Letort Spring Run,42041,Cumberland County,PA
211101,NaN,NaN,NaN,NaN,NaN,CMC,1992-11-01,NaN,NaN,NaN,...,NaN,NaN,NaN,PA,POINT (-77.16883 40.21418),20503050404,Letort Spring Run,42041,Cumberland County,PA
211102,NaN,NaN,NaN,NaN,NaN,CMC,1992-11-01,NaN,NaN,NaN,...,NaN,NaN,NaN,PA,POINT (-77.16003000000001 40.21847),20503050404,Letort Spring Run,42041,Cumberland County,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22151,NaN,NaN,NaN,NaN,NaN,CMC,2020-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,MD,POINT (-77.263728 38.4691169),20700110106,Tank Creek-Potomac River,24017,Charles County,MD
22152,NaN,NaN,NaN,NaN,NaN,CMC,2020-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,MD,POINT (-77.2694645 38.459693),20700110106,Tank Creek-Potomac River,24017,Charles County,MD
22153,NaN,NaN,NaN,NaN,NaN,CMC,2020-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,MD,POINT (-77.018505 38.784099),20700100307,Little Hunting Creek-Potomac River,24033,Prince George's County,MD
66594,NaN,NaN,NaN,NaN,NaN,CMC,2020-07-08,NaN,NaN,NaN,...,NaN,NaN,NaN,MD,POINT (-77.018505 38.784099),20700100307,Little Hunting Creek-Potomac River,24033,Prince George's County,MD


In [58]:
df[df["Parameter"] == "DO"]["MeasureValue"].mean()

7.895909943695156

In [66]:
df[df["Parameter"] == "DO.8"]["MeasureValue"].mean()

8.214134271784452

In [67]:
df

,Agency,BiasPC,CloudCover,Comments,Cruise,Database,Date,FieldActivityEventType,FlowStage,GaugeHeight,...,Weather Conditions Yesterday,WindDirection,WindSpeed,state,Point,HUC12_,HUCNAME_,FIPS_,COUNTY_,STATE_
0,NaN,NaN,NaN,NaN,NaN,CMC,2018-04-28,NaN,NaN,NaN,...,NaN,W,1-10 knots,MD,POINT (-75.93002800000002 39.480194),20600020206,Bohemia River,24015,Cecil County,MD
1,NaN,NaN,NaN,NaN,NaN,CMC,2018-04-28,NaN,NaN,NaN,...,NaN,W,1-10 knots,MD,POINT (-75.902528 39.480444),20600020206,Bohemia River,24015,Cecil County,MD
2,NaN,NaN,NaN,NaN,NaN,CMC,2018-04-28,NaN,NaN,NaN,...,NaN,S,1-10 knots,MD,POINT (-75.873639 39.466889),20600020206,Bohemia River,24015,Cecil County,MD
3,NaN,NaN,NaN,NaN,NaN,CMC,2018-04-28,NaN,NaN,NaN,...,NaN,SW,1-10 knots,MD,POINT (-75.854444 39.454),20600020206,Bohemia River,24015,Cecil County,MD
4,NaN,NaN,NaN,NaN,NaN,CMC,2018-04-28,NaN,NaN,NaN,...,NaN,SW,1-10 knots,MD,POINT (-75.84352800000001 39.463667),20600020206,Bohemia River,24015,Cecil County,MD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708712,VADEQ,NaN,NaN,NaN,BAY684,CBP,2016-11-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,POINT (-76.33835999999999 36.90456),20802080304,Hampton Roads,51710,Norfolk city,VA
2708713,VADEQ,NaN,NaN,NaN,BAY684,CBP,2016-11-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,POINT (-76.33835999999999 36.90456),20802080304,Hampton Roads,51710,Norfolk city,VA
2708714,VADEQ,NaN,NaN,NaN,BAY684,CBP,2016-11-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,POINT (-76.33835999999999 36.90456),20802080304,Hampton Roads,51710,Norfolk city,VA
2708715,VADEQ,NaN,NaN,NaN,BAY684,CBP,2016-11-08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,POINT (-76.33835999999999 36.90456),20802080304,Hampton Roads,51710,Norfolk city,VA


In [47]:
un1 = df[df["Database"]=="CBP"]["Station"].unique()

In [42]:
un2 = df[df["Database"]=="CMC"]["StationCode"].unique()

In [45]:
un1.size + un2.size

2581

In [53]:
df['Station'] = np.where(df['Station'].isnull(), df['StationCode'], df['Station'])

In [6]:
full_value_counts(df, "TideStage")

There are 9 unique values with 581580 nan values making up 21.5%
1035114-Ebb Tide --38.21%
852334-Flood Tide --31.47%
88843-High Slack Tide --3.28%
86646-Low Slack Tide --3.20%
22072-Outgoing (Ebb) --0.81%
18112-High --0.67%
12612-Incoming (Flood) --0.47%
11404-Low --0.42%


In [ ]:
test = "outgoing (ebb)"
re.match()

In [34]:
x = df.copy()


In [35]:
full_value_counts(x[x["Database"]=="CMC"], "Water Surfaces")

There are 5 unique values with 211949 nan values making up 80.2%
25041-Calm --9.48%
23855-Ripple --9.03%
3172-Waves --1.20%
244-White Caps --0.09%


In [49]:
full_value_counts(df[df["Database"]=="CMC"], "Weather Conditions Today")

There are 23 unique values with 148540 nan values making up 56.2%
55586-Sunny --21.03%
25844-Overcast --9.78%
13028-Partly Cloudy --4.93%
11742-Partly cloudy --4.44%
3468-Rain --1.31%
1169-sunny --0.44%
896-Drizzle --0.34%
764-intermittent Rain --0.29%
576-partly cloudy --0.22%
555-SUNNY --0.21%
508-Intermittent Rain --0.19%
499-Intermittent rain --0.19%
451-overcast --0.17%
174-Fog/haze --0.07%
157-Snow --0.06%
136-Fog/Haze --0.05%
76-drizzle --0.03%
42-Overcast  --0.02%
22-SUNNY  --0.01%
13-rain --0.00%
8-Sunny  --0.00%
7-partly Cloudy --0.00%


In [64]:
full_value_counts(df[df["Database"]=="CMC"], "WindSpeed")

There are 7 unique values with 229469 nan values making up 86.8%
16722-Calm --6.33%
16368-1-10 knots --6.19%
1534-10-20 knots --0.58%
133-1-10 Knots --0.05%
29-10-20 Knots --0.01%
6-20-30 knots --0.00%


In [76]:
full_value_counts(df, "COUNTY_")

There are 7 unique values with 0 nan values making up 0.0%
1394677-MD --51.49%
1062340-VA --39.22%
152958-PA --5.65%
38230-DC --1.41%
30010-NY --1.11%
17340-WV --0.64%
13162-DE --0.49%


In [77]:
full_value_counts(df, "STATE_")

There are 7 unique values with 0 nan values making up 0.0%
1394677-MD --51.49%
1062340-VA --39.22%
152958-PA --5.65%
38230-DC --1.41%
30010-NY --1.11%
17340-WV --0.64%
13162-DE --0.49%


In [80]:
full_value_comparison(df, "STATE_", "state")

MD
There are 3 unique values with 1312747 nan values making up 94.1%
81873-MD --5.87%
57-PA --0.00%

DE
There are 2 unique values with 4124 nan values making up 31.3%
9038-DE --68.67%

DC
There are 2 unique values with 33293 nan values making up 87.1%
4937-DC --12.91%

PA
There are 2 unique values with 66880 nan values making up 43.7%
86078-PA --56.28%

VA
There are 8 unique values with 1002920 nan values making up 94.4%
58988-VA --5.55%
72-PA --0.01%
72-DE --0.01%
72-MD --0.01%
72-DC --0.01%
72-WV --0.01%
72-NY --0.01%

NY
There are 2 unique values with 7606 nan values making up 25.3%
22404-NY --74.66%

WV
There are 2 unique values with 16886 nan values making up 97.4%
454-WV --2.62%



In [12]:
df.iloc[0:25,0:12]

,Agency,BiasPC,CloudCover,Cruise,Database,Date,FieldActivityEventType,FlowStage,GaugeHeight,GroupCode,HUC12,Lab
0,NaN,NaN,NaN,NaN,CMC,2018-04-28 12:05:00,NaN,NaN,NaN,FB,NaN,NaN
1,NaN,NaN,NaN,NaN,CMC,2018-04-28 12:40:00,NaN,NaN,NaN,FB,NaN,NaN
2,NaN,NaN,NaN,NaN,CMC,2018-04-28 13:13:00,NaN,NaN,NaN,FB,NaN,NaN
3,NaN,NaN,NaN,NaN,CMC,2018-04-28 13:53:00,NaN,NaN,NaN,FB,NaN,NaN
4,NaN,NaN,NaN,NaN,CMC,2018-04-28 14:24:00,NaN,NaN,NaN,FB,NaN,NaN
5,NaN,NaN,NaN,NaN,CMC,2018-04-28 14:53:00,NaN,NaN,NaN,FB,NaN,NaN
6,NaN,NaN,NaN,NaN,CMC,2018-05-08 10:43:00,NaN,NaN,NaN,FB,NaN,NaN
7,NaN,NaN,NaN,NaN,CMC,2018-05-08 11:16:00,NaN,NaN,NaN,FB,NaN,NaN
8,NaN,NaN,NaN,NaN,CMC,2018-05-08 11:42:00,NaN,NaN,NaN,FB,NaN,NaN
9,NaN,NaN,NaN,NaN,CMC,2018-05-08 12:22:00,NaN,NaN,NaN,FB,NaN,NaN
